In [1]:
import pickle 
import numpy as np
import pandas as pd
import statistics
from PIL import Image
import os 
import random
import time
import matplotlib.pyplot as plt
import netaddr

In [2]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import dgl.nn as dglnn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
#pip install netaddr

In [4]:
#directory should change based on the device, for laptop it should be users//jalal for office desktop it is users//jghadermazi

In [5]:
wed_data = pd.read_pickle(r'C:\\Users\\Jalal\\Box\\GAN Project\\Sample data\\Wednesday_Packet_Level_Data.pkl')

In [6]:
df1 = wed_data.copy()

In [7]:
del wed_data

In [8]:
df1[['Payload']] = df1[['Payload']].replace('', '00')

In [9]:
df1

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499258820.014743000,23.60.139.27,192.168.10.17,80,47757,12035,1,0023ae9b956700c1b114eb310800,45000034a7b6400037062efd173c8b1bc0a80a11,0050ba8da529957b5d3f78e3801003ab15920000010108...,00,fwd,1.0,1.0
1,1499258820.039091000,23.60.139.27,192.168.10.17,80,47757,12035,1,0023ae9b956700c1b114eb310800,45000722a7b740003706280e173c8b1bc0a80a11,0050ba8da529957b5d3f78e3801803ab74250000010108...,485454502f312e3120323030204f4b0d0a536572766572...,bwd,1.0,1.0
2,1499258820.039116000,192.168.10.17,23.60.139.27,47757,80,12035,435,00c1b114eb310023ae9b95670800,45000034fcd740004006d0dbc0a80a11173c8b1b,ba8d00505d3f78e3a5299c698010010011280000010108...,00,fwd,1.0,1.0
3,1499258820.082489000,192.168.10.12,208.185.50.90,51774,443,11808,4580,00c1b114eb31b8ac6f3604e30800,4500002804414000400668c7c0a80a0cd0b9325a,ca3e01bb4c69a4a67cf3967050109eb472ea0000,000000000000,fwd,1.0,1.0
4,1499258820.084153000,192.168.10.17,23.208.97.23,37115,443,12034,349,00c1b114eb310023ae9b95670800,4500017a021640004006f3c7c0a80a1117d06117,90fb01bb96268b0ed08217e7801801356d9e0000010108...,1703030141000000000000000174b7a44755da49b50c35...,fwd,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11334902,1499276130.701758000,172.16.0.1,192.168.10.51,443,53199,243533,0,b8ac6f360ba800c1b114eb310800,4500003cc45240003f06007dac100001c0a80a33,01bbcfcf167451881e1ff3d6a012fe884e0a0000020405...,00,bwd,0.0,0.0
11334903,1499276130.702180000,172.16.0.1,192.168.10.51,443,53199,243533,1,b8ac6f360ba800c1b114eb310800,45000034034d40003f06c18aac100001c0a80a33,01bbcfcf167451891e1ff5db8010000879580000010108...,00,bwd,0.0,0.0
11334904,1499276130.702964000,172.16.0.1,192.168.10.51,443,53199,243533,1,b8ac6f360ba800c1b114eb310800,450000bd034e40003f06c100ac100001c0a80a33,01bbcfcf167451891e1ff5db801800084ed60000010108...,16030300510200004d0303587c9e338fe3870b8fd3c17c...,bwd,0.0,0.0
11334905,1499276130.710822000,172.16.0.1,192.168.10.51,443,53199,243533,138,b8ac6f360ba800c1b114eb310800,45000034034f40003f06c188ac100001c0a80a33,01bbcfcf167452121e1ffa1f80100008748a0000010108...,00,bwd,0.0,0.0


In [10]:
#benign    0
#dos slowloris    1
#dos slowhttptest   2
#dos hulk     3
#dos goldeneye    4
#heartbleed    5
#ftp patator    6
#ssh patator    7
#webattack bruteforce    8
#webattack xss     9
#webattack sql injection 10
#infiltration     11
#botnet     12
#portscan   13
#ddos     14

In [11]:
df1['Class Label'].value_counts()

0.0    8845368
3.0    2246818
4.0     106177
5.0      49296
1.0      47586
2.0      39662
Name: Class Label, dtype: int64

In [12]:
df = df1[df1['Class Label']== 5]

In [13]:
df = df.reset_index(inplace = False)

In [14]:
df

,index,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,2440243,1499271295.627099000,94.31.29.248,192.168.10.5,80,52010,224779,83817,b8ac6f360a8b00c1b114eb310800,450011441a6740003706d1885e1f1df8c0a80a05,0050cb2a67c97fad5a4d48ed5010002257fb0000,357de65045d57c7115ccb8bf13cde59922de45f30561e9...,fwd,1.0,5.0
1,2440244,1499271295.627288000,94.31.29.248,192.168.10.5,80,52010,224779,88197,b8ac6f360a8b00c1b114eb310800,450005dc1a6a40003706dced5e1f1df8c0a80a05,0050cb2a67c990c95a4d48ed50100022bfae0000,13c71b2f3022b864971b2049d41ba328ddd1926eaf94af...,bwd,1.0,5.0
2,2440245,1499271295.627481000,94.31.29.248,192.168.10.5,80,52010,224779,89657,b8ac6f360a8b00c1b114eb310800,45000b901a6b40003706d7385e1f1df8c0a80a05,0050cb2a67c9967d5a4d48ed5010002252470000,454cd645572a5d61306c938ddc608eb3afe984c298e9c1...,fwd,1.0,5.0
3,2440246,1499271295.627516000,192.168.10.5,94.31.29.248,52010,80,224779,1831,00c1b114eb31b8ac6f360a8b0800,450000284985400080066a86c0a80a055e1f1df8,cb2a00505a4d48ed67c99c3150100100f55f0000,000000000000,fwd,1.0,5.0
4,2440247,1499271295.627519000,192.168.10.5,94.31.29.248,52010,80,224779,1831,00c1b114eb31b8ac6f360a8b0800,450000284986400080066a85c0a80a055e1f1df8,cb2a00505a4d48ed67c9a79950100100e9f70000,000000000000,bwd,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49291,2489534,1499271575.694021000,192.168.10.17,54.77.135.34,34717,443,226035,709,00c1b114eb310023ae9b95670800,450000344d48400040066553c0a80a11364d8722,879d01bb5dbfb7074cc4eca780110152dc340000010108...,00,bwd,1.0,5.0
49292,2489535,1499271575.714789000,192.168.10.19,75.101.141.254,43163,443,226105,2675,00c1b114eb310023ae9badb30800,45000034f69e40004006a006c0a80a134b658dfe,a89b01bb3239051479e81d048010019502a40000010108...,00,bwd,1.0,5.0
49293,2489536,1499271575.715202000,185.86.137.17,192.168.10.17,443,53706,226025,4386,0023ae9b956700c1b114eb310800,450000347ea44000f306fbfdb9568911c0a80a11,01bbd1ca353612fa8406f2f68010174981160000010108...,00,bwd,1.0,5.0
49294,2489537,1499271575.728876000,192.168.10.14,23.15.155.27,54151,80,225687,424,00c1b114eb31b8ac6f3607ee0800,450000295ce6400080062108c0a80a0e170f9b1b,d38700500904f465a1369193501001002de70000,000000000000,bwd,1.0,5.0


In [15]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    
    output = np.abs(output.astype(float))/255
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [16]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    output = np.abs(output.astype(float))/255
    
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [17]:
df['pkt_header_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], x['TCP Header Array'])), axis=1)

In [18]:
new_cols = ['Source IP', 'Dst IP', 'Source Port', 'Dst Port','Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_header_values']
df = df[new_cols]

In [19]:
# 70% of benign data for train, 30% for test benign

In [20]:
#random.seed(0)
#mlist = random.sample(streams_lst, 61492)

In [21]:
#len(imlist)

In [22]:
#df_train = df.loc[df['Stream Index'].isin(imlist)]
#df_train = df_train.reset_index(inplace = False)

In [23]:
#df_test = df.loc[~df['Stream Index'].isin(imlist)]
#df_test = df_test.reset_index(inplace = False)

In [24]:
#streams_lst1 = list(df_test['Stream Index'].unique())

In [25]:
#len(streams_lst1)

In [26]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,94.31.29.248,192.168.10.5,80,52010,1499271295.627099000,224779,83817,fwd,"[0.06666666666666667, 0.26666666666666666, 0.1..."
1,94.31.29.248,192.168.10.5,80,52010,1499271295.627288000,224779,88197,bwd,"[0.0196078431372549, 0.8627450980392157, 0.101..."
2,94.31.29.248,192.168.10.5,80,52010,1499271295.627481000,224779,89657,fwd,"[0.043137254901960784, 0.5647058823529412, 0.1..."
3,192.168.10.5,94.31.29.248,52010,80,1499271295.627516000,224779,1831,fwd,"[0.0, 0.1568627450980392, 0.28627450980392155,..."
4,192.168.10.5,94.31.29.248,52010,80,1499271295.627519000,224779,1831,bwd,"[0.0, 0.1568627450980392, 0.28627450980392155,..."
...,...,...,...,...,...,...,...,...,...
49291,192.168.10.17,54.77.135.34,34717,443,1499271575.694021000,226035,709,bwd,"[0.0, 0.20392156862745098, 0.30196078431372547..."
49292,192.168.10.19,75.101.141.254,43163,443,1499271575.714789000,226105,2675,bwd,"[0.0, 0.20392156862745098, 0.9647058823529412,..."
49293,185.86.137.17,192.168.10.17,443,53706,1499271575.715202000,226025,4386,bwd,"[0.0, 0.20392156862745098, 0.49411764705882355..."
49294,192.168.10.14,23.15.155.27,54151,80,1499271575.728876000,225687,424,bwd,"[0.0, 0.1607843137254902, 0.3607843137254902, ..."


In [27]:
streams_lst = list(df['Stream Index'].unique())
print('Total number of flows in class:', len(streams_lst))    
random.shuffle(streams_lst)

# Directory path where to save the CSV files
directory_path = 'Wednesday_heartbleed_all/'

for PKT in range(2, 16):
    
    file_name = f'wednesday_heartbleed_{PKT}pkt.csv'
    file_path = directory_path + file_name
    

    data = []
    num_pkt = PKT
    pkt_val_pad = [0] * 25


    for j in range(len(streams_lst)):

        df_stream = df[df['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)


        if len(df_stream) >= num_pkt:

            src_node = (int(netaddr.IPAddress(df_stream['Source IP'][0])), int(df_stream['Source Port'][0]))
            dst_node = (int(netaddr.IPAddress(df_stream['Dst IP'][0])), int(df_stream['Dst Port'][0]))
            stream_no = df_stream['Stream Index'][0]

            pkt_val_seq = []

            for k in range(0, num_pkt):

                pkt_val = []
                if df_stream['Direction'][k] == 'fwd':
                    pkt_val = [*df_stream['pkt_header_values'][k], *pkt_val_pad]
                else:
                    pkt_val = [*pkt_val_pad, *df_stream['pkt_header_values'][k]]

                pkt_val_seq.extend(pkt_val)

            data.append([src_node, dst_node, stream_no, pkt_val_seq])

    df2 = pd.DataFrame(data, columns= ['src_node', 'dst_node', 'stream_number', 'pkt_vals'])
    
    df2.to_csv(file_path, index=False)

Total number of flows in class: 1656
